In [276]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import wikipedia

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-night-mode-clientpref-0 vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Indigenous languages of the Americas - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main

In [429]:
url_link = "https://en.wikipedia.org/wiki/Indigenous_languages_of_the_Americas"
content = requests.get(url_link).text
soup = BeautifulSoup(content,'html.parser')
table = soup.find_all('table', 'wikitable')

df= pd.read_html(str(table))
df=pd.DataFrame(df[0])

df = df.drop('Source',axis=1)
data = df.rename(columns={'Number of speakers':'number'})

In [430]:
def fixxr(x):
    x = x.split(' ',1)[0]
    x = x.replace(',','')
    return x

data.number=data.number.apply(fixxr)
#Remove abnormalities (beta)
for num in data.number:
    test = any(ele not in ['1','2','3','4','5','6','7','8','9','0'] for ele in list(num))
    if test== True:
        print(list(data.number).index(num))
data.number[90] = '10500'

data = data[data.number!= '0']
data.number = data.number.apply(int)
# data = data[data.number >= 10000]
# data['Number of speakers'].apply(.split(' ',1)[0])
data

90


,Language,number,Official Recognition,Area(s) Language is spoken
0,Guaraní,6500000,Paraguay (Official Language) Bolivia Corriente...,"Paraguay, Bolivia, Argentina, Brazil"
1,Southern Quechua,5000000,Bolivia (Official Language) Peru (Official Lan...,"Bolivia, Peru, Argentina, Chile"
2,Nahuatl,1700000,Mexico,Mexico
3,Aymara,1700000,Bolivia (Official Language) Peru (Official Lan...,"Bolivia, Peru, Chile"
4,Qʼeqchiʼ,1100000,Guatemala Belize Mexico,"Guatemala, Belize, Mexico"
...,...,...,...,...
229,Comanche,9,NaN,US
230,Carijona,6,"Colombia (Amazonas, Guaviare)","Colombia (Amazonas, Guaviare)"
231,Itonama,5,Bolivia,Bolivia
232,Kiliwa,4,Mexico,Mexico


In [447]:
table = soup.find('table', 'wikitable')
links = table.findAll('a')
for link in links: # Remove all the non titles
    if 'title' not in str(link):
        links.remove(link)

#Everything past this point is very incomplete

for link in links: # find categories and select only languages
    cats = wikipedia.page(link.get('title')).categories


# links_content = {}
# print('Done extracting links. About to fetch: %s links..' % len(links))
# for link in links:
#     print('Fetching: %s' % link)
#     links_content[link] = requests.get(r'https://en.wikipedia.org/wiki/{}'.format(link.get('href'))).text

KeyboardInterrupt: 

In [340]:
r'https://en.wikipedia.org/wiki/{}'.format(links[3].get('href'))

'https://en.wikipedia.org/wiki//wiki/Mato_Grosso_do_Sul'

In [440]:
links[9]

<a href="/wiki/Aymara_language" title="Aymara language">Aymara</a>

In [446]:
thing = wikipedia.page(links[9].get('title'))
'Languages of' in thing.categories[24]

True

In [424]:
links[100].get('title')

IndexError: list index out of range

In [374]:
links[7].get('title')

'Jujuy Province'